In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import lightgbm as lgb
import xgboost as xgb
from tensorflow import keras

In [ ]:
# Cargar datos
train = pd.read_csv("../../../data/post_cleaning/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/post_cleaning/validation_set.csv", parse_dates=["date"])
test_set = pd.read_csv("../../../data/post_cleaning/test_set.csv", parse_dates=['date'])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values
X_test = test_set.drop(columns=['target_trend','date']).values
y_test = test_set['target_trend'].values

In [ ]:


# Importar la clase ModelLoader
from model_loader import ModelLoader

# Leer el archivo de métricas
metrics_path = '../../score_models/model_comparison_metrics.csv'
df = pd.read_csv(metrics_path, index_col='model')

# Encontrar el modelo con el mayor trend_change_f1_score
best_model_name = df['trend_change_f1_score'].idxmax()
print(f"Mejor modelo según trend_change_f1_score: {best_model_name}")

# Construir la ruta del archivo del modelo
model_file = f'../../score_models/{best_model_name}.txt'

# Instanciar y cargar el modelo
loader = ModelLoader(best_model_name, model_file)
model = loader.load()




In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
window_size = 5
n_test = X_test.shape[0]
y_pred_test = []

# Paso 1: Entrena con train + val
X_train_current = np.vstack([X_train, X_val])
y_train_current = np.concatenate([y_train, y_val])

# Instanciar el modelo
#model = instantiate_model(best_model_name) No es necesario porque se carga en load
model.fit(X_train_current, y_train_current)

for start in tqdm(range(0, n_test, window_size), desc="Backtesting"):
    end = min(start + window_size, n_test)
    
    # Paso 2: Predice los siguientes window_size días del test
    X_pred = X_test[start:end]
    y_pred = model.predict(X_pred)
    y_pred_test.extend(y_pred)
    print(f"Predicción del bloque {start} a {end}: {y_pred}")
    # Paso 3: Reentrena si quedan días por predecir
    if end < n_test:
        # Elimina los primeros window_size días del entrenamiento actual
        X_train_current = np.vstack([X_train_current[window_size:], X_test[start:end]])
        y_train_current = np.concatenate([y_train_current[window_size:], y_test[start:end]])
        
        # Vuelve a separar en train y validation (por ejemplo, 80% train, 20% val)
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
            X_train_current, y_train_current, test_size=0.2, random_state=42
        )
        
        # Reinstancia y reentrena el modelo con el nuevo train+val
        #model = instantiate_model(best_model_name) deberia guardar y luego volver a cargar aqui?
        model.fit(np.vstack([X_train_split, X_val_split]), np.concatenate([y_train_split, y_val_split]))
        print(f"Reentrenamiento realizado con datos hasta el día {end}")
print("Predicciones rolling window sobre el test:")
print(y_pred_test)

AGREGADO POR GABRIEL

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Cargar datos
test_set = pd.read_csv("../../../data/post_cleaning/test_set.csv", parse_dates=['date'])
X_test = test_set.drop(columns=['target_trend','date']).values
y_test = test_set['target_trend'].values

# Cargar el mejor modelo previamente entrenado
from model_loader import ModelLoader
metrics_path = '../../score_models/model_comparison_metrics.csv'
df = pd.read_csv(metrics_path, index_col='model')
best_model_name = df['trend_change_f1_score'].idxmax()
print(f"Mejor modelo según trend_change_f1_score: {best_model_name}")
model_file = f'../../score_models/{best_model_name}.txt'
loader = ModelLoader(best_model_name, model_file)
model = loader.load()

# Realizar predicciones sobre el test
if best_model_name.lower() == "multiclass_neural_network":
    y_pred_test = np.argmax(model.predict(X_test), axis=1)
    # Si usas mapeo inverso, descomenta:
    # y_pred_test = np.vectorize(inv_map.get)(y_pred_test)
else:
    y_pred_test = model.predict(X_test)

# Crear DataFrame para backtesting
backtest_df = test_set.copy()
backtest_df['signal'] = y_pred_test

# Seleccionar la columna de precio (ajusta si tu columna es diferente)
price_col = [col for col in test_set.columns if 'open_d0' in col]
if price_col:
    price_col = price_col[-1]
else:
    price_col = test_set.columns[1]  # fallback: segunda columna

# Simulación de estrategia
initial_capital = 10000.0
cash = initial_capital
position = 0.0  # Unidades de BTC
portfolio_values = []

for i, row in tqdm(backtest_df.iterrows(), total=len(backtest_df), desc="Simulación Backtesting"):
    price = row[price_col]
    signal = row['signal']
    # Lógica de trading
    if signal == 1 and cash > 0:  # Señal de compra
        position = cash / price
        cash = 0.0
    elif signal == -1 and position > 0:  # Señal de venta
        cash = position * price
        position = 0.0
    # Calcular el valor actual del portafolio
    current_portfolio_value = cash + position * price
    portfolio_values.append(current_portfolio_value)

backtest_df['portfolio_value'] = portfolio_values

# Métricas de rendimiento
final_portfolio_value = backtest_df['portfolio_value'].iloc[-1]
total_return_pct = (final_portfolio_value - initial_capital) / initial_capital * 100
buy_and_hold_return_pct = (backtest_df[price_col].iloc[-1] - backtest_df[price_col].iloc[0]) / backtest_df[price_col].iloc[0] * 100

print("--- Resultados del Backtesting ---")
print(f"Capital Inicial: ${initial_capital:,.2f}")
print(f"Valor Final del Portafolio: ${final_portfolio_value:,.2f}")
print(f"Retorno Total de la Estrategia: {total_return_pct:.2f}%")
print(f"Retorno de Comprar y Mantener (Buy & Hold): {buy_and_hold_return_pct:.2f}%")

# Visualización
plt.figure(figsize=(14, 7))
plt.plot(backtest_df['date'], backtest_df['portfolio_value'], label='Estrategia del Modelo', color='blue')
buy_and_hold_values = (backtest_df[price_col] / backtest_df[price_col].iloc[0]) * initial_capital
plt.plot(backtest_df['date'], buy_and_hold_values, label='Comprar y Mantener (Buy & Hold)', color='orange', linestyle='--')
plt.title('Rendimiento de la Estrategia vs. Buy & Hold')
plt.xlabel('Fecha')
plt.ylabel('Valor del Portafolio ($)')
plt.legend()
plt.grid(True)
plt.show()

Aquí tienes el código para realizar el backtesting siguiendo el esquema de la celda 20 de multiclass_neural_network.ipynb, pero adaptado para tu flujo de selección de modelo en select_best_model.ipynb.
El código usa el modelo previamente entrenado y guardado, predice sobre el test, genera señales y simula la estrategia, mostrando métricas y gráficos comparativos.

Notas:

El código es compatible con cualquier modelo que hayas guardado y cargado con ModelLoader.
Si tu modelo de red neuronal usa mapeo inverso de clases, ajusta la línea de y_pred_test.
Cambia price_col si tu columna de precios tiene otro nombre.
Se usa tqdm para mostrar el progreso de la simulación.
¿Quieres agregar métricas adicionales o guardar los resultados?